# Projct Name- Movie Recommendation System

### Import Standard Lybraries

In [2]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Data Collection and Pre-Processing

In [3]:
# Load the dataset
data=pd.read_csv('Movies.csv')
data

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [4]:
# Read first 5 rows of the Dataset
data.head(5)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [5]:
# Number o columns in the datset
data.shape

(4803, 24)

In [6]:
# selecting the relevent feature for recommendation
selected_features=['genres','keywords','tagline','cast','director']
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [7]:
# replacing the null values with null string

for feature in selected_features:
    data[feature]=data[feature].fillna('')

In [8]:
# combining all the 5 selected features
combined_features=data['genres']+''+data['keywords']+''+data['tagline']+''+data['cast']+''+['director']
combined_features

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

In [9]:
# converting the the text data to feature vectors
vectorizer = TfidfVectorizer()

In [10]:
feature_vectors= vectorizer.fit_transform(combined_features)
type(feature_vectors)

scipy.sparse.csr.csr_matrix

In [11]:
print(feature_vectors)

  (0, 18398)	0.2564421528740414
  (0, 14726)	0.15957063169340832
  (0, 12964)	0.24359089788232116
  (0, 20663)	0.16439436569049035
  (0, 23622)	0.1996521668495562
  (0, 19966)	0.20543086585470333
  (0, 19110)	0.21908457116762534
  (0, 24444)	0.20100109785818776
  (0, 24134)	0.2387050774899074
  (0, 19137)	0.15919992460570997
  (0, 16465)	0.26556026339870364
  (0, 16103)	0.08835340168579521
  (0, 24113)	0.12829726013599882
  (0, 21495)	0.0764355993614407
  (0, 20224)	0.2784115183904239
  (0, 4160)	0.24936959688746826
  (0, 23429)	0.13276875966544502
  (0, 20350)	0.3422858971059385
  (0, 9246)	0.16807328414828726
  (0, 3968)	0.22162153236593868
  (0, 8437)	0.2784115183904239
  (0, 19409)	0.10132314315076281
  (0, 8099)	0.12002440222760764
  (0, 432)	0.0926082265468152
  (0, 243)	0.07989251742674608
  :	:
  (4801, 4896)	0.33188071652182954
  (4801, 19707)	0.33188071652182954
  (4801, 24299)	0.33188071652182954
  (4801, 24424)	0.31656136572946825
  (4801, 7350)	0.2750534117813915
  (4801, 

### Cosine Similarity

In [12]:
# getting the similarity scores using cosine similarity
similarity=cosine_similarity(feature_vectors)

In [13]:
print(similarity)

[[1.         0.07125681 0.01547801 ... 0.         0.         0.        ]
 [0.07125681 1.         0.02915734 ... 0.01287857 0.         0.        ]
 [0.01547801 0.02915734 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01287857 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [14]:
similarity.shape

(4803, 4803)

In [25]:
# getting the movie name from the user
movie_name= input('Enter your favourate movie name:')

Enter your favourate movie name:spider


In [26]:
# Creating a list with all the movies names given in the dataset
list_of_all_titles=data['title'].tolist()
list_of_all_titles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [27]:
# Finding the close match for the movie name given by the user
find_close_match=difflib.get_close_matches(movie_name,list_of_all_titles)
find_close_match

['Spider', 'Casper', 'Spider-Man']

In [28]:
close_match=find_close_match[0]
print(close_match)

Spider


In [29]:
# Finding the index of the movie with title
index_of_the_movie=data[data.title == close_match]['index'].values[0]
index_of_the_movie

3085

In [30]:
# getting a list of similar movies

similarity_score= list(enumerate(similarity[index_of_the_movie]))
similarity_score

[(0, 0.0),
 (1, 0.0),
 (2, 0.06557803436508727),
 (3, 0.0047645693684005225),
 (4, 0.033939599474589),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.009305607398053852),
 (13, 0.0),
 (14, 0.0),
 (15, 0.02928366068031042),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.05240652754645418),
 (22, 0.0),
 (23, 0.0),
 (24, 0.004016365079655467),
 (25, 0.004377993475944842),
 (26, 0.0),
 (27, 0.03157801113297557),
 (28, 0.008873742975870179),
 (29, 0.08587104765028672),
 (30, 0.0),
 (31, 0.0),
 (32, 0.027812931528553582),
 (33, 0.0),
 (34, 0.011676492389434972),
 (35, 0.009706364177217618),
 (36, 0.022191234614655303),
 (37, 0.023857226537583494),
 (38, 0.0),
 (39, 0.0),
 (40, 0.012258882565667944),
 (41, 0.0),
 (42, 0.0),
 (43, 0.0),
 (44, 0.0),
 (45, 0.004060230869824354),
 (46, 0.0),
 (47, 0.0),
 (48, 0.0),
 (49, 0.023602640506377018),
 (50, 0.0),
 (51, 0.0),
 (52, 0.010030104274635528),
 (53, 0.0),
 (54, 0.0),
 (55, 0.017534956

In [31]:
len(similarity_score)

4803

In [32]:
# sorting the movie basedon their similarity score
sorted_similar_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)
sorted_similar_movies

[(3085, 1.0000000000000004),
 (2494, 0.21318924511497633),
 (3894, 0.1843040303444342),
 (822, 0.11862933350348162),
 (1508, 0.11467174993911831),
 (3225, 0.11460145409261911),
 (3838, 0.11146339081185715),
 (1519, 0.11112553685333135),
 (3757, 0.10889008083556077),
 (4748, 0.10209222671252921),
 (626, 0.10101508458285433),
 (2842, 0.09777876207432053),
 (3734, 0.09648147227254339),
 (346, 0.0950166473660995),
 (2008, 0.09390148457624783),
 (3511, 0.09369243953445945),
 (3073, 0.09297918473144319),
 (1052, 0.09175022257942676),
 (4603, 0.09156605495530529),
 (3412, 0.0913869404149969),
 (1380, 0.09078678148304481),
 (2961, 0.09065213005429487),
 (494, 0.09015102293889643),
 (1619, 0.08995349403332147),
 (286, 0.08939117003804366),
 (4011, 0.08804651345049522),
 (29, 0.08587104765028672),
 (1254, 0.08544314381929168),
 (1457, 0.08539385079841266),
 (1665, 0.08531050260695491),
 (1809, 0.08471229893492019),
 (4234, 0.08442121629560888),
 (2366, 0.08304101084068558),
 (698, 0.083021739285

In [33]:
# print the name of similar movies based on the index

i=1
for movie in sorted_similar_movies:
    index=movie[0]
    title_from_index=data[data.index==index]['title'].values[0]
    if(i<30):
        print(i,title_from_index)
        i+=1

1 Spider
2 The White Countess
3 A Room with a View
4 Maid in Manhattan
5 The Number 23
6 Limbo
7 Gods and Monsters
8 Bogus
9 The Cooler
10 Absentia
11 The X Files
12 Stir of Echoes
13 The Crying Game
14 What Lies Beneath
15 In the Valley of Elah
16 Videodrome
17 Romeo Is Bleeding
18 Training Day
19 My Dog Tulip
20 Shade
21 The Man in the Iron Mask
22 Jindabyne
23 The Lion King
24 The Prince & Me
25 Eraser
26 Halloween III: Season of the Witch
27 Skyfall
28 Get Carter
29 Smilla's Sense of Snow


In [38]:
movie_name=input("Enter your favourite movie")
list_of_all_titles=data['title'].tolist()

find_close_match=difflib.get_close_matches(movie_name,list_of_all_titles)

close_match=find_close_match[0]

index_of_the_movie=data[data.title==close_match]['index'].values[0]

sorted_similar_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)
print('Movies suggested for you : \n')

i=1

for movie in sorted_similar_movies:
    index=movie[0]
    title_from_index=data[data.index == index]['title'].values[0]
    if (i<30):
        print(i, title_from_index)
        i+=1

Enter your favourite moviebatman
Movies suggested for you : 

1 Spider
2 The White Countess
3 A Room with a View
4 Maid in Manhattan
5 The Number 23
6 Limbo
7 Gods and Monsters
8 Bogus
9 The Cooler
10 Absentia
11 The X Files
12 Stir of Echoes
13 The Crying Game
14 What Lies Beneath
15 In the Valley of Elah
16 Videodrome
17 Romeo Is Bleeding
18 Training Day
19 My Dog Tulip
20 Shade
21 The Man in the Iron Mask
22 Jindabyne
23 The Lion King
24 The Prince & Me
25 Eraser
26 Halloween III: Season of the Witch
27 Skyfall
28 Get Carter
29 Smilla's Sense of Snow
